In [1]:
import os
import json
import pandas as pd
import plotly.graph_objects as go

In [2]:
data_dir = './data_dump'
pd.set_option('display.max_columns', None)

In [3]:
for fn in os.listdir(data_dir):
  with open(os.path.join(data_dir, fn), 'r') as f:
    data = json.load(f)
    df_raw = pd.json_normalize(data)

In [4]:
static_data = {}
static_cols = ['angular_velocity', 'position', 'velocity', 'rotation']
for col in df_raw.columns:
  if len(df_raw[col].unique()) == 1 and all(stc not in col for stc in static_cols):
    static_data[col] = df_raw[col].unique()[0]
df = df_raw.drop(list(static_data.keys()), axis=1)
static_data

{'medium.base_density': np.float64(1.225),
 'medium.drag_coef': np.float64(0.47),
 'medium.drag_model': 'quadratic',
 'medium.molar_mass': np.float64(0.029),
 'medium.rha_coef': np.float64(1e-05),
 'medium.speed_of_sound': np.float64(343.0),
 'medium.type': 'gas',
 'medium.viscosity': np.float64(1.81e-05),
 'naming.ammo_name': '5.56x45_M855',
 'naming.projectile_uid': 'proj_ZAuKc4a2',
 'naming.weapon_name': 'SCAR_L_CQC',
 'projectile.ammo': '<Resource#-9223371996220357260>',
 'projectile.caliber': np.float64(5.7),
 'projectile.core_caliber': np.float64(5.4),
 'projectile.core_hardness': np.float64(0.6),
 'projectile.core_mass': np.float64(2.0),
 'projectile.cross_section': np.float64(2.55175863287831e-05),
 'projectile.drag_coef': np.float64(0.28),
 'projectile.fragmentation_chance': np.float64(0.1),
 'projectile.fragmentation_count': np.int64(0),
 'projectile.fragments_max': np.int64(4),
 'projectile.fragments_min': np.int64(2),
 'projectile.impact_count': np.int64(0),
 'projectile.ki

In [36]:
df['projectile.velocity.length'] = (df['projectile.velocity.x']**2 + df['projectile.velocity.y']**2 + df['projectile.velocity.z']**2)**0.5
df['projectile.position.length'] = (df['projectile.position.x']**2 + df['projectile.position.y']**2 + df['projectile.position.z']**2)**0.5
df.columns

Index(['projectile.angular_velocity.x', 'projectile.angular_velocity.y',
       'projectile.angular_velocity.z', 'projectile.current_drag_coef',
       'projectile.effective_cross_section', 'projectile.flight_time',
       'projectile.mach_number', 'projectile.position.x',
       'projectile.position.y', 'projectile.position.z',
       'projectile.rotation.x.x', 'projectile.rotation.x.y',
       'projectile.rotation.x.z', 'projectile.rotation.y.x',
       'projectile.rotation.y.y', 'projectile.rotation.y.z',
       'projectile.rotation.z.x', 'projectile.rotation.z.y',
       'projectile.rotation.z.z', 'projectile.state', 'projectile.velocity.x',
       'projectile.velocity.y', 'projectile.velocity.z', 'timing.exec_ratio',
       'timing.exec_time', 'timing.frame', 'timing.time',
       'projectile.velocity.length', 'projectile.position.length'],
      dtype='object')

In [93]:
fig = go.Figure()

mach_min = df['projectile.mach_number'].min()
mach_max = df['projectile.mach_number'].max()
mach_threshold = 1.0
mach_threshold_norm = (mach_threshold - mach_min) / (mach_max - mach_min)
mach_colorscale = [
  [0.0, '#2f9'],
  [mach_threshold_norm, '#4af'],
  [mach_threshold_norm, '#f00'],
  [1.0, '#ff0']
]

fig.add_trace(go.Scatter3d(
  x=df['projectile.position.x'],
  z=df['projectile.position.y'],
  y=df['projectile.position.z'],
  mode='lines+markers',
  marker=dict(
    size=3,
    color=df['projectile.current_drag_coef'],
    colorscale='Jet',
    colorbar=dict(
      title='Drag Coefficient',
      y=0.0,
      len=0.5
    )
  ),
  line=dict(
    width=4,
    color=df['projectile.mach_number'],
    colorscale=mach_colorscale,
    colorbar=dict(
      title='Mach Number',
      y=0.5,
      len=0.5
    ),
  ),
  name='Position',
  customdata=pd.concat([
    df[[
    'timing.frame',
    'projectile.flight_time',
    'projectile.velocity.z',  # Rate Z
    'projectile.velocity.y',  # Rate Y
    'projectile.velocity.x',  # Rate X
    'projectile.velocity.length',  # Vel m/s
    'projectile.mach_number',      # Vel M
    'projectile.current_drag_coef' # Cd
  ]],
    df[[
      'projectile.position.z',  # Dist (Z)
      'projectile.position.y',  # Drop (Y)
      'projectile.position.x',  # Drift (X)
    ]].abs()
  ], axis=1),
  hovertemplate=(
    "Frame %{customdata[0]} (%{customdata[1]:.5f}s)<br>" +
    "Dist: %{customdata[8]:.2f} m | Drop: %{customdata[9]:.2f} m | Drift: %{customdata[10]:.2f} m<br>" +
    "Rate: (%{customdata[2]:.2f}, %{customdata[3]:.2f}, %{customdata[4]:.2f}) m/s<br>" +
    "Velocity: %{customdata[5]:.2f} m/s (%{customdata[6]:.2f} M)<br>" +
    "Cd: %{customdata[7]:.3f}" +
    "<extra></extra>"
  )
))


scene_ax = dict(
  backgroundcolor='#444',
  gridcolor='#666',
  showbackground=True,
  zerolinecolor='#666',
  color='white'
)
fig.update_layout(
  paper_bgcolor='#333',
  plot_bgcolor='#444',
  font=dict(
    color='white'
  ),
  scene_camera=dict(
    eye=dict(x=-2.0, y=0.0, z=0.0)  # положение "глаза камеры"
  ),
  scene=dict(
    aspectmode='manual',
    aspectratio=dict(x=1.0, y=2.5, z=0.5),
    xaxis=scene_ax,
    yaxis=scene_ax,
    zaxis=scene_ax
  ),
  margin=dict(l=0, r=0, b=0, t=10)
)

In [ ]:
pd.concat([
  df[[
    'timing.frame',
    'projectile.flight_time',
    'projectile.velocity.z',  # Rate Z
    'projectile.velocity.y',  # Rate Y
    'projectile.velocity.x',  # Rate X
    'projectile.velocity.length',  # Vel m/s
    'projectile.mach_number',      # Vel M
    'projectile.current_drag_coef' # Cd
  ]],
  df[[
    'projectile.position.z',  # Dist (Z)
    'projectile.position.y',  # Drop (Y)
    'projectile.position.x',  # Drift (X)
    ]].abs()
  ], axis=1)

,timing.frame,projectile.flight_time,projectile.velocity.z,projectile.velocity.y,projectile.velocity.x,projectile.velocity.length,projectile.mach_number,projectile.current_drag_coef,projectile.position.z,projectile.position.y,projectile.position.x
0,0,0.000000,-922.000000,0.000000,0.0,922.000000,0.000000,0.000000,0.000000,0.000000,0.0
1,1,0.016667,-901.771667,-0.163144,0.0,901.771682,2.688047,0.252000,15.029529,0.002719,0.0
2,2,0.033333,-882.591125,-0.322818,0.0,882.591185,2.629072,0.374531,29.739382,0.008099,0.0
3,3,0.050000,-864.372192,-0.479298,0.0,864.372325,2.573152,0.371228,44.145584,0.016088,0.0
4,4,0.066667,-847.038208,-0.632830,0.0,847.038444,2.520036,0.368097,58.262886,0.026635,0.0
5,5,0.083333,-830.520874,-0.783633,0.0,830.521244,2.469500,0.365122,72.104904,0.039695,0.0
6,6,0.100000,-814.758972,-0.931905,0.0,814.759505,2.421345,0.362292,85.684219,0.055227,0.0
7,7,0.116667,-799.697632,-1.077822,0.0,799.698358,2.375392,0.359595,99.012512,0.073191,0.0
8,8,0.133333,-785.287354,-1.221544,0.0,785.288304,2.331482,0.357022,112.100632,0.093550,0.0
9,9,0.150000,-771.483337,-1.363215,0.0,771.484542,2.289470,0.354563,124.958687,0.116270,0.0
